# T5 + doc2query - Indexing

Author: Monique Monteiro (moniquelouise@gmail.com)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Libraries installation

In [2]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyserini-0.21.0-py3-none-any.whl (154.1 MB)
  Using cached onnxruntime-1.14.1-cp39-cp39-manylinux_2_27_x86_64.whl (5.0 MB)
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 52.1 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.98-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.7 MB/s eta 0:00:00
  Using cached pyjnius-1.4.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
  Using cached nmslib-2.1.1-cp39-cp39-manylinux2010_x86_64.whl (13.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━

In [4]:
!pip install -q condacolab

In [5]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [6]:
!conda install faiss-cpu -c pytorch

Solving environment: | / - \ | / - \ | / - \ | / - \ done

# All requested packages already installed.



In [7]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install trectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 15.6 MB/s eta 0:00:00
  Created wheel for trectools: filename=trectools-0.0.49-py3-none-any.whl size=27141 sha256=3c2d8f6085173bae81054dc86a5fc9ccc5f0e36030e4092102581fd14a52e977
  Stored in directory:

## Index building

In [ ]:
from pyserini.search.lucene import LuceneSearcher

In [ ]:
#beir-v1.0.0-trec-covid.flat
LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:33, 6.85MB/s]                           


In [ ]:
!ls ~/.cache/pyserini/indexes/

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


## Dataset download and preparation



In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-04-10 22:42:37--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 13.249.85.69, 13.249.85.127, 13.249.85.92, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1681425758&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9u

In [9]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-6-3"

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [10]:
import json

with open(f'{main_dir}/trec-covid/queries.tsv','w') as f_out:
  with open(f'{main_dir}/trec-covid/queries.jsonl', 'r') as f:
    for line in f:
      data = json.loads(line)
      id = data['_id']
      text = data['text']
      f_out.write(f'{id}\t{text}\n')
        


In [11]:
topics = "trec-covid"

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-04-10 22:44:42--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 13.249.85.69, 13.249.85.16, 13.249.85.127, ...
Connecting to huggingface.co (huggingface.co)|13.249.85.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  --.-KB/s    in 0.07s   

2023-04-10 22:44:42 (13.7 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

In [12]:
import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

## Searching with BM25 the original index

In [13]:
!python -m pyserini.search.lucene \
  --index ~/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Unrecognized index name /root/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade


In [ ]:
run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [ ]:
from evaluate import load
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [ ]:
results['NDCG@10']

0.5963435398557583

## Searching with BM25 the expanded index

In [14]:
#index_name = "expanded_trec_covid"
#index_name = "expanded_trec_covid_3_queries"
#index_name = "expanded_trec_covid_20_queries"
index_name = "expanded_trec_covid_5_queries_expand_title"

In [24]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name} \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-expanded_trec_covid_5_queries_expand_title.txt...
100% 50/50 [00:04<00:00, 11.52it/s]


In [25]:
run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned-{index_name}.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [26]:
from evaluate import load
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [27]:
results['NDCG@10']

0.7030050774722006

## Removal of documents without text

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jsonlines

corpus = []
docs_to_remove_expansion = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
    for item in reader:
        if len(item["text"].strip()) == 0:
          docs_to_remove_expansion[item["_id"]] = item["title"]

In [ ]:
len(docs_to_remove_expansion)

42140

In [ ]:
#trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping
#index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping"
#index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_20_queries"
#index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries"
index_name = "trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16"

In [ ]:
collections_path = f"{main_dir}/collections/{index_name}"
collections_path

'/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16'

In [ ]:
import glob
import os
from tqdm.auto import tqdm

def remove_expansions_from_empty_docs(index_name):
  collections_path = f"{main_dir}/collections/{index_name}"
  print(collections_path)
  new_collections_path = collections_path + "-expand-non-empty"
  extension = "*.json"
  files = glob.glob(os.path.join(collections_path, extension))
  print(files)

  for file in tqdm(files):
    with jsonlines.open(file) as reader:
      # Loop through each line in the file
      itens = []
      for item in reader:
        if item["id"] in docs_to_remove_expansion:
          itens.append({"id":item["id"], "contents":docs_to_remove_expansion[item["id"]]})
        else:
          itens.append(item)
      with jsonlines.open(new_collections_path + "/" + file[file.rfind('/')+1:], mode='w') as writer:
        for item in itens:
          writer.write(item)
        print(new_collections_path + "/" + file[file.rfind('/')+1:])

  return new_collections_path

In [ ]:
new_collections_path = remove_expansions_from_empty_docs(index_name)

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16
['/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_1.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_2.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_3.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_4.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_5.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_6.json', '/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16/json_7.json', '/content/gdrive/MyDri

  0%|          | 0/11 [00:00<?, ?it/s]

/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_1.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_2.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_3.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_4.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_5.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_6.json
/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty/json_7.json
/content/gdrive/MyDrive/Unicamp-au

In [ ]:
new_collections_path, index_name

('/content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty',
 'trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16')

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {new_collections_path} \
  --index {main_dir}/{index_name}-expand-non-empty \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-13 00:36:15,556 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-13 00:36:15,560 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-13 00:36:15,560 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-13 00:36:15,561 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty
2023-04-13 00:36:15,561 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-13 00:36:15,562 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-13 00:36:15,562 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-13 00:36:15,563 INFO  [main] index.IndexCollectio

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.96it/s]


In [ ]:
from evaluate import load

def eval(run_file):
  run = pd.read_csv(run_file, sep="\s+", 
                    names=["query", "q0", "docid", "rank", "score", "system"])
  run = run.to_dict(orient="list")

  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10']

In [ ]:
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty.txt")

0.6821625236566535

In [ ]:
eval("/content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-no-fp16-expand-non-empty.txt")

0.6821625236566535

## BM25 hyperparameters tuning for the best model up to now

In [ ]:
#k1 in [0.5, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]
#b in [0.25, 0.5, 0.75, 1.0]

In [ ]:
collections_path = f'{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty'
index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries'

Recreates the "best" index in the appropriate location (mounted gdrive).

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {collections_path} \
  --index {main_dir}/{index_name}-expand-non-empty \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-12 20:57:58,299 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-12 20:57:58,301 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-12 20:57:58,301 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-12 20:57:58,302 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty
2023-04-12 20:57:58,302 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-12 20:57:58,302 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-12 20:57:58,302 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-12 20:57:58,303 INFO  [main

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt...
100% 50/50 [00:04<00:00, 10.63it/s]


In [ ]:
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")

0.6559657499563727

### Grid search

In [ ]:
#index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries'
index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping'

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.31it/s]


In [ ]:
print(eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt"))
print(eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt"))

0.539745045579447
0.5980067077187501


In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.5 --b 0.25

Setting BM25 parameters: k1=0.5, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.59it/s]


In [ ]:
print(eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt"))
print(eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt"))

0.539745045579447
0.5980067077187501


In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.5 --b 0.5

Setting BM25 parameters: k1=0.5, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:09<00:00,  5.07it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.5980067077187501

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.5 --b 0.75

Setting BM25 parameters: k1=0.5, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.74it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6156444131414522

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 0.5 --b 1

Setting BM25 parameters: k1=0.5, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:12<00:00,  4.06it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6057718066558745

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1 --b 0.25

Setting BM25 parameters: k1=1.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.71it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6637250087365776

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1 --b 0.5

Setting BM25 parameters: k1=1.0, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:15<00:00,  3.17it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6604301584685243

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1 --b 0.75

Setting BM25 parameters: k1=1.0, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:15<00:00,  3.27it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6682028573917758

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1 --b 1

Setting BM25 parameters: k1=1.0, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:12<00:00,  3.97it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.623947446377171

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.2 --b 0.25

Setting BM25 parameters: k1=1.2, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.72it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6715429947061176

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.2 --b 0.5

Setting BM25 parameters: k1=1.2, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:15<00:00,  3.19it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.675344210851204

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.2 --b 0.75

Setting BM25 parameters: k1=1.2, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.91it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6635389843616403

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.2 --b 1

Setting BM25 parameters: k1=1.2, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.35it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6228957397249061

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.5 --b 0.25

Setting BM25 parameters: k1=1.5, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.61it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6826970109791185

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.5 --b 0.5

Setting BM25 parameters: k1=1.5, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:12<00:00,  4.10it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6766700068170876

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.5 --b 0.75

Setting BM25 parameters: k1=1.5, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.54it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6684199108693945

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 1.5 --b 1

Setting BM25 parameters: k1=1.5, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.61it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6161271263446961

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2 --b 0.25

Setting BM25 parameters: k1=2.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.79it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6919870918421648

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2 --b 0.5

Setting BM25 parameters: k1=2.0, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.49it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6811640691738177

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2 --b 0.75

Setting BM25 parameters: k1=2.0, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:08<00:00,  5.64it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6719667725621191

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2 --b 1

Setting BM25 parameters: k1=2.0, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:08<00:00,  5.82it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6002627716455143

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2.5 --b 0.25

Setting BM25 parameters: k1=2.5, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.52it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6853357466910411

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2.5 --b 0.5

Setting BM25 parameters: k1=2.5, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.59it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6827955569276699

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2.5 --b 0.75

Setting BM25 parameters: k1=2.5, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:15<00:00,  3.17it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6711342299872449

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 2.5 --b 1

Setting BM25 parameters: k1=2.5, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:09<00:00,  5.10it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.5767086647535795

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 3 --b 0.25

Setting BM25 parameters: k1=3.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:12<00:00,  4.16it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6881795090313019

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 3 --b 0.5

Setting BM25 parameters: k1=3.0, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:10<00:00,  4.75it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6827531443250856

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 3 --b 0.75

Setting BM25 parameters: k1=3.0, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.24it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6669107299931841

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 3 --b 1

Setting BM25 parameters: k1=3.0, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.19it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.5583313540583201

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:14<00:00,  3.44it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6940007375216084

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.5

Setting BM25 parameters: k1=4.0, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:15<00:00,  3.21it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6767039167465603

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.75

Setting BM25 parameters: k1=4.0, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.35it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6633777379779217

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 1

Setting BM25 parameters: k1=4.0, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:09<00:00,  5.48it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.5116709446089776

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 5 --b 0.25

Setting BM25 parameters: k1=5.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.39it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6866087049133124

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 5 --b 0.5

Setting BM25 parameters: k1=5.0, b=0.5
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:11<00:00,  4.36it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6783843260029286

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 5 --b 0.75

Setting BM25 parameters: k1=5.0, b=0.75
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.63it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6541996581687293

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 5 --b 1

Setting BM25 parameters: k1=5.0, b=1.0
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:13<00:00,  3.74it/s]


In [ ]:
#eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_18_queries-expand-non-empty.txt")
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.47501733376264754

### Comparing with baseline

In [ ]:
!python -m pyserini.search.lucene \
  --index ~/.cache/pyserini/indexes/lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.57b812594b11d064a23123137ae7dade \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned.txt...
100% 50/50 [00:10<00:00,  4.82it/s]


In [ ]:
run = pd.read_csv(f"{main_dir}/runs/run.{topics}.bm25tuned.txt", sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [ ]:
from evaluate import load
trec_eval = load("trec_eval")
results = trec_eval.compute(predictions=[run], references=[qrel])

In [ ]:
results['NDCG@10']

0.6410048418087232

### Analysis for 3 queries

In [ ]:
collections_path = f'{main_dir}/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries'
index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries'

Recreates the index in the appropriate location (mounted gdrive).

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {collections_path} \
  --index {main_dir}/{index_name} \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-04-12 22:36:46,938 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-04-12 22:36:46,940 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-04-12 22:36:46,940 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-04-12 22:36:46,940 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/gdrive/MyDrive/Unicamp-aula-6-3/collections/trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries
2023-04-12 22:36:46,940 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-04-12 22:36:46,941 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-04-12 22:36:46,941 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-04-12 22:36:46,941 INFO  [main] index.IndexColle

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name} \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries.txt...
100% 50/50 [00:12<00:00,  4.01it/s]


In [ ]:
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping_3_queries.txt")

0.6845000508941358

### Analysis for 5 queries

In [ ]:
#trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping

In [ ]:
index_name = 'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping'

In [ ]:
index_name

'trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping'

In [ ]:
!python -m pyserini.search.lucene \
  --index {main_dir}/{index_name}-expand-non-empty \
  --topics {main_dir}/{topics}/queries.tsv \
  --output {main_dir}/runs/run.{topics}.bm25tuned-{index_name}-expand-non-empty.txt \
  --hits 1000 \
  --bm25 --k1 4 --b 0.25

Setting BM25 parameters: k1=4.0, b=0.25
Running /content/gdrive/MyDrive/Unicamp-aula-6-3/trec-covid/queries.tsv topics, saving to /content/gdrive/MyDrive/Unicamp-aula-6-3/runs/run.trec-covid.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt...
100% 50/50 [00:12<00:00,  3.86it/s]


In [ ]:
eval(f"{main_dir}/runs/run.{topics}.bm25tuned-trec_covid_doc2query-adafactor-bs-32-split-1000-bf16-early-stoping-expand-non-empty.txt")

0.6940007375216084